<table style="width:100%; background-color: #EBF5FB">
  <tr>
    <td style="border: 1px solid #CFCFCF">
      <b>Household data: Processing Notebook</b>
      <ul>
        <li><a href="main.ipynb">Main Notebook</a></li>
        <li>Processing Notebook</li>
      </ul>
      <br>This Notebook is part of the <a href="http://data.open-power-system-data.org/household_data">Household Data Package</a> of <a href="http://open-power-system-data.org">Open Power System Data</a>.
    </td>
  </tr>
</table>

# Table of Contents
* [1. Introductory Notes](#1.-Introductory-Notes)
* [2. Settings](#2.-Settings)
	* [2.1 Import Python libraries](#2.1-Import-Python-libraries)
	* [2.2 Set version number and recent changes](#2.2-Set-version-number-and-recent-changes)
	* [2.3 Select timerange](#2.3-Select-timerange)
	* [2.4 Select download source](#2.4-Select-download-source)
* [3. Download](#3.-Download)
* [4. Read](#4.-Read)
	* [4.1 Preparations](#4.1-Preparations)
	* [4.2 Select household subset](#4.2-Select-household-subset)
	* [4.3 Reading loop](#4.3-Reading-loop)


# 1. Introductory Notes

This Notebook handles missing data, performs calculations and aggragations and creates the output files.

# 2. Settings

## 2.1 Import Python libraries

This section: load libraries and set up a log.

In [ ]:
# Python modules
from datetime import datetime, date, timedelta, time
import pandas as pd
import numpy as np
import logging
import json
import sqlite3
import yaml
import itertools
import os
import pytz
import hashlib
from shutil import copyfile

# Scripts from household repository package
from household.download import download
from household.read import read

# Reload modules with execution of any code, to avoid having to restart
# the kernel after editing timeseries_scripts
%load_ext autoreload
%autoreload 2

households_yaml_path = 'conf/households.yml'

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger('log')
# For more detailed logging messages, replace 'INFO' with 'DEBUG'
# (May slow down computation).
#logger.setLevel('INFO')

## 2.2 Set version number and recent changes

Executing this script till the end will create a new version of the data package.
The Version number specifies the local directory for the data <br>
We include a note on what has been changed.

In [ ]:
version = '2017-06-30'
data_path = os.path.join(version, 'original_data')
changes = '''Initial upload'''

## 2.3 Select timerange

Select the time range to read and process data. <br>
*Default: all data.*

Type `None` to process all available data.

In [ ]:
start_from_user = None  # i.e. date(2016, 1, 1)
end_from_user = None  # i.e. date(2016, 1, 31)

## 2.4 Select download source

The raw data can be downloaded as a zip file from the OPSD Server. To do this, specify an archive version to use, that has been cached on the OPSD server as input.

In [ ]:
archive_version = '2017-06-30' # i.e. '2017-06-30'

# 3. Download

This section: download raw data to process.

If the original data does not exist, it will be downloaded from the OPSD Server and extracted in a local directory

In [ ]:
download(version=archive_version)

# 4. Read

This section: Read each downloaded file into a pandas-DataFrame and merge data from different sources if it has the same time resolution.

## 4.1 Preparations

Set the title of the rows at the top of the data used to store metadata internally. The order of this list determines the order of the levels in the resulting output.

In [ ]:
headers = ['project', 'region', 'household', 'type', 'feed']

Create a dictionary of empty DataFrames to be populated by the read function

In [ ]:
data_sets = {'1min': pd.DataFrame(), '15min': pd.DataFrame()}

## 4.2 Select household subset

Optionally, specify a subset of households to process. <br>
The next cell prints the available sources and datasets.

In [ ]:
with open(households_yaml_path, 'r') as f:
    households = yaml.load(f.read())
for k, v in households.items():
    print(yaml.dump({k: list(v['feeds'].keys())}, default_flow_style=False))

Copy from its output and paste to following cell to get the right format.

Type `subset = None` to include all data.

In [ ]:
subset = yaml.load('''
    insert_household_here:
    - insert_feed1_here
    - insert_feed2_here
    more_households_here:
    - more_feeds_here
    ''')

subset = None

Now eliminate households and feeds not in subset.

In [ ]:
if subset:  # eliminate households and feeds not in subset
    households = {household_name: {k: v
                                   for k, v in households[household_name].items()
                                   for k, v in households[household_name]['feeds'].items()
                                   if k in feed_list}
                  for household_name, feed_list in subset.items()}

## 4.3 Reading loop

Loop through households and feeds to do the reading

In [ ]:
# For each source in the household dictionary
for household_name, household_dict in households.items():
    df = read(household_name, household_dict['dir'], household_dict['feeds'], 
              household_dict['project'], household_dict['region'], household_dict['type'], headers, 
              out_path=data_path,
              start_from_user=start_from_user,
              end_from_user=end_from_user)
    